In [1]:
%%capture
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import os
import sys

import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
# I recommend that you do not change things in this cell!
url = "https://nomad-hzb-ce.de/nomad-oasis/api/v1"
token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']

sys.path.insert(1, '../../../python-scripts-c6fxKDJrSsWp1xCxON1Y7g')
from api_calls import *
from ce_nome_tool_box import *
from cp_plots_functions import *
sys.path.append(os.path.abspath('..'))
from helpers import *

# CP Overview

This notebook is designed for uploads with multiple `CE_NOME_Chronopotentiometry` files.
It allows you to get an overview of all concatenated CP files and optionally you can inspect specific time intervalls. 

**For more custom plots you can copy and edit the file `cp_plots_jupyter.ipynb` in your own upload.**


In [3]:
# all widgets used in notebook
uploads = get_all_uploads(url, token, number_of_uploads=200)
upload_names = widgets.Dropdown(options=[u.get("upload_name","--no-name--") for u in uploads],description='Upload:')

time_slider = widgets.IntRangeSlider(
    step=1,
    description='Time Range (in s):',
    continuous_update=False,
    layout=widgets.Layout(width='500px'),
    style={'description_width': 'initial'}
)

overview_output = widgets.Output()
interval_output = widgets.Output()

In [4]:
# widget logic
def set_slider(minvalue, maxvalue, values):
    if time_slider.max < minvalue:
        time_slider.max = maxvalue
        time_slider.min = minvalue
    else:
        time_slider.min = minvalue
        time_slider.max = maxvalue
    time_slider.value = values

def on_change_uploads(change):
    global upload_name, df
    if change['type'] == 'change' and change['name'] == 'value':
        upload_name = change['new']
        with overview_output:
            overview_output.clear_output()
            df = run_analysis(url, token, upload_name)
        with interval_output:
            if df.empty:
                set_slider(minvalue=0, maxvalue=0, values=[0,0])
                interval_output.clear_output()
            else:
                set_slider(df['time'].min(), df['time'].max(), [df['time'].min(), df['time'].max()])         

def inspect_interval(change):
    with interval_output:
        interval_output.clear_output()
        start_time, end_time = time_slider.value
        if not df.empty:
            filtered_df = df[(df['time'] >= start_time) & (df['time'] <= end_time)]
            plot_interval(filtered_df)
            display(filtered_df)
        
upload_names.observe(on_change_uploads)
time_slider.observe(inspect_interval, names='value')

### Overview

In [5]:
# trigger on_change_uploads once in the beginning
on_change_uploads({'new': upload_names.value,
                   'type': 'change',
                   'name': 'value',
                  })

# show widgets
display(widgets.HBox([upload_names]), overview_output)

Output()

### Inspect Time Intervals

In [6]:
#initial trigger
inspect_interval(None)

display(time_slider, interval_output)

IntRangeSlider(value=(0, 0), continuous_update=False, description='Time Range (in s):', layout=Layout(width='5…

Output()

### Download Overview Data as csv Table

In [7]:
download_button = widgets.Button(description='Prepare Data for Download',
                                 layout=widgets.Layout(width='200px'))
download_button_output = widgets.Output()

def on_download_button_clicked(b):
    file_name = f'{upload_name}_cp_overview.csv'
    with download_button_output:
        clear_output(wait=True)
        try:
            create_zip_download_link([df], [file_name], 'cp_overview')
            plot_list = [plt.figure(fig_num) for fig_num in plt.get_fignums()]
            fig_names = ['overview', 'timeinterval']
            create_zip_plots_link(plot_list, fig_names, 'cp_plots')
        except NameError:
            print('Please select another upload that contains CP data.')

download_button.on_click(on_download_button_clicked)
display(download_button, download_button_output)

Button(description='Prepare Data for Download', layout=Layout(width='200px'), style=ButtonStyle())

Output()